In [1]:
import requests
import pandas as pd

In [2]:
def _extract_edamam_products_from_result(r, cur_list=[], limit=10):
    for t in ["parsed", "hints"]:
        if t in list(r.keys()):
            for i in range(0, len(r[t])):
                if "food" in list(r[t][i].keys()):
                    food_elm = r[t][i]["food"]
                    k = list(food_elm.keys())
                    if "foodId" in k and "label" in k and "category" in k and "foodContentsLabel" in k and "image" in k:
                        cur_list.append(
                            {
                                "foodId": food_elm["foodId"],
                                "label": food_elm["label"],
                                "category": food_elm["category"],
                                "foodContentsLabel": food_elm["foodContentsLabel"],
                                "image": food_elm["image"]
                            }
                        )
                        if len(cur_list)>=limit:
                            return cur_list

    if "_links" in list(r.keys()) and "next" in list(r["_links"].keys()) and "href" in list(r["_links"]["next"].keys()):
        print(f"""calling {r["_links"]["next"]["href"]}""")
        response_next = requests.get(r["_links"]["next"]["href"])
        if response_next.status_code!=200:
            print(f"Erreur code {response_next.status_code}: {response_next.reason}")
        else:
            cur_list = _extract_edamam_products_from_result(response_next.json(), cur_list, limit)

    return cur_list
    
    
def search_edamam_products(q="champagne", limit=10):
    url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"
    
    querystring = {
        "ingr": q,
    }
    
    headers = {
    	"X-RapidAPI-Key": "c242fd3e15msha16ff277efdd3b5p1641a2jsn4c5a07611de0",
    	"X-RapidAPI-Host": "edamam-food-and-grocery-database.p.rapidapi.com"
    }

    ret = []
    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code!=200:
        print(f"Erreur code {response.status_code}: {response.reason}")
    else:
        ret = _extract_edamam_products_from_result(response.json(), cur_list = ret, limit=limit)
    return ret


In [3]:
q = input("Entrez le type de produit que vous voulez chercher (par défault 'champagne'):").strip()
if q=="":
    q = "champagne"

limit = 10
while True:
    limit = input("Entrez le nombre de produits à retenir (10 par défaut): ").strip()
    if limit=="":
        limit = 10
    try:
        limit = int(limit)
        if limit > 0:
            break
    except ValueError:
        print("Veuillez entrer un nombre entier...")  
        continue
      
r = search_edamam_products(q, limit)

Entrez le type de produit que vous voulez chercher (par défault 'champagne'):champagne
Entrez le nombre de produits à retenir (10 par défaut): 10
calling https://api.edamam.com/api/food-database/v2/parser?session=40&app_key=69d004c3e3c5b967261e625baea627fc&app_id=85b409cf&ingr=champagne
calling https://api.edamam.com/api/food-database/v2/parser?session=80&app_key=69d004c3e3c5b967261e625baea627fc&app_id=85b409cf&ingr=champagne
calling https://api.edamam.com/api/food-database/v2/parser?session=120&app_key=69d004c3e3c5b967261e625baea627fc&app_id=85b409cf&ingr=champagne


In [4]:
df = pd.DataFrame(r)

In [5]:
df

,foodId,label,category,foodContentsLabel,image
0,food_b3dyababjo54xobm6r8jzbghjgqe,"Champagne Vinaigrette, Champagne",Packaged foods,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...,https://www.edamam.com/food-img/d88/d88b64d973...
1,food_bmu5dmkazwuvpaa5prh1daa8jxs0,"Champagne Dressing, Champagne",Packaged foods,SOYBEAN OIL; WHITE WINE (PRESERVED WITH SULFIT...,https://www.edamam.com/food-img/ab2/ab2459fc2a...
2,food_bu12urpbtuo9v6b4jpvk2a1fh4hh,"Champagne Simply Dressed Vinaigrette, Champagne",Packaged foods,FILTERED WATER; CANOLA OIL; CHAMPAGNE AND WHIT...,https://www.edamam.com/food-img/736/736a3e27a6...
3,food_bu12urpbtuo9v6b4jpvk2a1fh4hh,"Champagne Simply Dressed Vinaigrette, Champagne",Packaged foods,FILTERED WATER; CANOLA OIL; CHAMPAGNE AND WHIT...,https://www.edamam.com/food-img/736/736a3e27a6...
4,food_b48c55sagj89z4afsne5dar76h4x,Champagne Vinegar,Packaged foods,CHAMPAGNE VINEGAR DILUTED WITH WATER TO 7% ACI...,https://www.edamam.com/food-img/ad8/ad8c8d6ba8...
5,food_bb0nrsgbser5g4ac2enxib1deyh8,Champagne Mustard,Packaged foods,DIJON MUSTARD (VINEGAR; WATER; MUSTARD SEED; S...,https://www.edamam.com/food-img/775/775b39c0b0...
6,food_arm1rlyb5v81v6arurmjqb6xiy9t,"Light Champagne Dressing, Light Champagne",Packaged foods,WATER; SOYBEAN OIL; WHITE WINE (PRESERVED WITH...,https://www.edamam.com/food-img/ee0/ee0475f45b...
7,food_bzb3i0lbxkz4nnbv10dsladd965e,Inglenook Champagne,Packaged foods,Dealcoholized Champagne; Water; Grape Juice Co...,https://www.edamam.com/food-img/cb3/cb33b008db...
8,food_byl67wcbbfw82ua6j1n7oa6ago4a,Cola Champagne,Packaged foods,CARBONATED WATER; HIGH FRUCTOSE CORN SYRUP; AR...,https://www.edamam.com/food-img/f82/f82d164536...
9,food_b3iaz2gav39j6abdpavjka0qgz2e,Cola Champagne,Packaged foods,CARBONATED WATER; HIGH FRUCTOSE CORN SYRUP; AR...,https://www.edamam.com/food-img/b4b/b4b747a25b...


In [62]:
df.to_csv("champagne.csv")